In [1]:
import numpy as np
import pandas as pd
import warnings
import sys

# Ignorer les warnings
warnings.filterwarnings('ignore')

# Fixer seed pour toujours avoir les valeurs aléatoires
np.random.seed(2)

features = ['ref_cli', 'prov', 'reg', 'com', 'fkt', 'date_dep_dmd', 'date_val_dmd',
            'date_int', 'tar', 'cat', 'pce']

cp_trn = np.arange(101, 120)
cp_tve = np.arange(501, 519)
cp_mjg = np.arange(401, 421)
cp_fnr = np.arange(301, 324)
cp_die = np.arange(202, 210)
cp_tul = np.arange(601, 622)

cp = dict()
cp['Antananarivo'] = cp_trn


# cp['Toamasina'] = cp_tve
# cp['Mahajanga'] = cp_mjg
# cp['Fianarantsoa'] = cp_fnr
# cp['Antsiranana'] = cp_die
# cp['Toliara'] = cp_tul


### Contrainte & conditions: 
    - Depuis 2000 à 2023
    - de janvier à décembre
    - du 1 à 30/31 du mois
    - Entre mois de Mai à octobre: augmentation
    - pic au mois d'Aout
    - Random de 1 à 10 (nombre de demandes déposées) 

In [2]:
# Liste des province, region, district, commune
data_df = pd.read_csv('pro_reg_dist_com.csv', delimiter=';')
data_df = data_df.drop('N°', axis=1)

In [3]:
# Nettoyage des données
df_copied = data_df.copy()

# Remplacement des valeurs '\\N' par null
df_copied.replace('\\N', np.nan, inplace=True)

# Remplacement des valeurs 'Unknown Name' par null
df_copied.replace('Unknown Name', np.nan, inplace=True)

# Suppression de toutes les lignes contenant les valeurs nulles
df_copied.dropna(inplace=True)

# Réinitialistion de l'index après suppression
fkt_df = df_copied.reset_index(drop=True)
fkt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17121 entries, 0 to 17120
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fokontany  17121 non-null  object
 1   Kaomina    17121 non-null  object
 2   Distrika   17121 non-null  object
 3   Region     17121 non-null  object
 4   Province   17121 non-null  object
dtypes: object(5)
memory usage: 668.9+ KB


D'après les informations affichées, il y a 17121 observations dans le data, lesquelles sont organisées en Provinces, Regions, Districts, Communes et Fokontany ou quartier.  
Mais ce qui nous interesse c'est de récupérer seulement la liste des districts de la région d'Analamanga de la province d'Antananarivo. L'idée c'est de travailler sur un échantillon un peu réduit pour éviter les calculs trop important.

In [4]:
# Filtre par Région d'Analamanga
analamanga_df = fkt_df[fkt_df['Region'] == 'Analamanga']
analamanga_df.reset_index(drop=True)

# Récupération de la liste de districts
districts = analamanga_df['Distrika'].unique()
districts

array(['Andramasina', 'Antananarivo-Avaradrano',
       'Antananarivo-Atsimondrano', 'Manjakandriana', 'Ambohidratrimo',
       'Anjozorobe', 'Ankazobe'], dtype=object)

In [5]:
import random

# Création des données
years = range(2000, 2024)

data = {'district': [],
        'nb_dmd': [],
        'energy': [],
        'date_dep_dmd': []}

for year in years:
    for month in range(1, 13):
        for district in districts:
            # Génération de la demande en électricité avec une saisonnalité et une tendance progressive
            demand = 200 + 10 * (year - 2000) + np.sin(month * np.pi / 6) * 20
            demand = int(demand)
            # Ajout d'une variation chaque année
            demand += random.randint(-20, 20)
            # Pique au mois de juillet
            if month == 7:
                demand += random.randint(30, 50)

            # Génération de la puissance énergétique avec une saisonnalité et une tendance progressive
            power = 100 + 5 * (year - 2000) + np.cos(month * np.pi / 6) * 15
            # Ajout d'une variation chaque année
            power += random.randint(-10, 10)
            # Pique au mois de juillet
            if month == 7:
                power += random.randint(20, 30)

            data['district'].append(district)
            data['nb_dmd'].append(demand)
            data['energy'].append(power)
            data['date_dep_dmd'].append(pd.Timestamp(year, month, 1))

# Création du dataframe
df = pd.DataFrame(data)

df

,district,nb_dmd,energy,date_dep_dmd
0,Andramasina,223,114.990381,2000-01-01
1,Antananarivo-Avaradrano,228,111.990381,2000-01-01
2,Antananarivo-Atsimondrano,192,104.990381,2000-01-01
3,Manjakandriana,215,109.990381,2000-01-01
4,Ambohidratrimo,220,111.990381,2000-01-01
...,...,...,...,...
2011,Antananarivo-Atsimondrano,440,224.000000,2023-12-01
2012,Manjakandriana,446,222.000000,2023-12-01
2013,Ambohidratrimo,449,220.000000,2023-12-01
2014,Anjozorobe,442,230.000000,2023-12-01


**date_dep_dmd**: c'est la date de dépôt de la demande. C'est l'index du Dataframe.  

**district**: Nom des disctricts

**nb_dmd**: Nombre de demandes déposées

**energy**: Puissance d'energie, en KWh, équivalente au nombre de demande.

On sauvegarde les données aux format 'csv' pour une utilisation ultérieure

In [6]:
df.to_csv('energy_antananarivo.csv', index=False)